<a href="https://colab.research.google.com/github/johnathan2012/Programming-iOS-Book-Examples/blob/master/flagchat4_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# flagchat4 套件用法示範

本套件主要將 OpenAI Chat API 抽象化, 納入串流、function calling 功能, 並且將介面統一使用生成器產生回覆, 不論是否啟用串流模式, 都可用一致的方式取得回覆。另外, 也搭配 function calling 設計一個簡易的外掛系統。

## 事前準備

In [ ]:
!pip install openai
!pip install googlesearch-python
from googlesearch import search
from google.colab import userdata
import openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Preparing metadata (setup.py) ... done
  Created wheel for googlesearch-python: filename=googlesearch_python-1.2.3-py3-none-any.whl size=4209 sha256=d2cd613d52284bc7c2b698eabd2656e696b2d17c4b92f832bee299252ef95102
  Stored in directory: /root/.cache/pip/wheels/98/24/e9/6c225502948c629b01cc895f86406819281ef0da385f3eb669
Successfully built googlesearch-python


## 下載套件

In [ ]:
%%bash
git clone https://github.com/FlagTech/flagchat4.git flagchat4
cd flagchat4/
git pull

Already up to date.


Cloning into 'flagchat4'...


## 從模組匯入工具函式

In [ ]:
from flagchat4 import (
    set_client,       # 設定要使用的用戶端物件 (預設直接使用 openai 模組)
    get_reply,        # 輸入訊息串列傳回回覆
    chat,             # 輸入 system, user 發言取得回覆並會記錄對答歷史
    tools_table,      # 記錄可用工具函式的參考表, 預設有 Google 搜尋函式
    set_backtrace,    # 設定記錄幾組對答 (預設：2)
    empty_history,    # 清除對答歷史
)

flagchat4 預設為直接使用 openai 模組當用戶端物件, 你也可以透過 `set_client` 函式設定客製的用戶端物件。

In [ ]:
# 預設使用環境變數 OPENAI_API_KEY
openai.api_key = userdata.get('OPENAI_API_KEY')

# 也可以使用客製的用戶端物件
# client = openai.OpenAI(userdata.get('OPENAI_API_KEY'))
# set_client(client)

In [ ]:
set_backtrace(2)

2

## 單一問答測試

get_reply 可以會透過 function calling 機制使用 func_table 傳入的函式表格傳回回覆。模組內預設的 tools_table 只有 Google 搜尋函式。

```python
get_reply(
    messages, # 訊息串列
    stream=False # 是否啟用串流模式
    tools_table=None # 工具函式參考表
)
```
另外有選用的參數：

```python
model='gpt-3.5-turbo' # 指定模型
debug=False # 是否要顯示除錯訊息, 包含訊息串列內容
```

In [ ]:
# 測試非串流方式 function_calling 功能
for chunk in get_reply(       # 不論是否串流回覆, 都以生成器統一函式介面
    [{"role":"user", "content":"2023 金馬獎最佳男配角？"}], # 訊息串列
    tools_table=tools_table): # 工具函式表
    print(chunk)              # 非串流模式只會生成一次

嘗試叫用：google_res(**{'user_msg': '2023金馬獎最佳男配角'})
2023年金馬獎最佳男配角的得主是陳慕義，他因在電影《老狐狸》中的表現獲此殊榮。


In [ ]:
# 測試串流方式 function_calling 功能
for chunk in get_reply(   # 不論是否串流回覆, 都以生成器統一函式介面
    [{"role":"user", "content":"2023 金馬獎最佳女配角？"}], # 訊息串列
    stream=True,              # 啟用串流模式
    tools_table=tools_table): # 工具函式表
    print(chunk, end='')  # 串流方式每次生成片段, 不換行才能接續內容

嘗試叫用：google_res(**{'user_msg': '2023金馬獎最佳導演'})
根據上述的搜尋結果摘要，2023年金馬獎最佳導演得主是《老狐狸》的蕭雅全。

In [ ]:
# 測試非串流、無 function calling 功能
for chunk in get_reply(
    [{"role":"user", "content":"2023 金馬獎最佳導演是誰？"}]):
    print(chunk)

很抱歉，我無法提供當前或實時的數據信息，包括最新的獎項得主。截至我知识更新的時間在2023年之前，因此我沒有這一年度最佳導演獎的結果。要獲得最新的信息，建議查看最近的電影獎項結果，例如奧斯卡獎（Academy Awards）、金球獎（Golden Globe Awards）或其他電影節獎項的官方網站或可靠新聞來源。


In [ ]:
# 測試串流、無 function calling 功能
for chunk in get_reply(
    [{"role":"user", "content":"2023 金馬獎影帝是誰？"}],
    stream=True):
    print(chunk, end='')

很抱歉，我無法提供即時信息或最新事件的更新，因為我的知識截止日期是2023年的初期。關於2023年金馬獎影帝的獲獎者，您可能需要查閱最新的新聞報導或官方金馬獎的公告以獲得最新資訊。

## 歷史紀錄測試

chat 是以 get_reply 函式為基礎, 加上對談歷史紀錄的功能, 可以使用 backtrace 設定要記錄的對談組數。

```python
chat(
    sys_msg, # system 角色發言
    user_msg, # user 角色發言
    stream=False, # 是否啟用串流模式
    tools_table=tools_table # 工具函式參考表 (預設是模組內建的參考表)
)
```
一樣可以使用選用的參數：

```python
model='gpt-3.5-turbo' # 指定模型
debug=False # 是否要顯示除錯訊息, 包含訊息串列內容
```

In [ ]:
for chunk in chat(
    '小助理',                   # system 角色發言
    '2023 金馬獎最佳女配角是誰？', # user 角色發言
    True):                     # 使用串流模式
    print(chunk, end='')

嘗試叫用：google_res(**{'user_msg': '2023 金馬獎 最佳女配角'})
2023年金馬獎最佳女配角是方志友，她憑藉在電影《本日公休》中的表演獲得該獎項。

底下會因為有歷史紀錄而影響建議的搜尋關鍵字：

In [ ]:
for chunk in chat(
    '小助理',                 # system 角色發言
    '那 2022 呢？',           # user 角色發言 (會延續對答脈絡)
    True):                   # 使用串流模式
    print(chunk, end='')

嘗試叫用：google_res(**{'user_msg': '2022金馬獎最佳女配角'})
2022年金馬獎最佳女配角得主是林詹珍妹，她因在電影《哈勇家》中的表演而獲此殊榮。

chat 會使用模組內預設的 func_table, 如果不想啟用, 可以加讓 func_table 參數值 None

In [ ]:
for chunk in chat(
    '小助理',               # system 角色發言
    '那 2021 呢？',         # user 角色發言
    True,                  # 串流模式
    None):                 # 不使用工具函式參考表 (因此不會搜尋)
    print(chunk, end='')

您好！看起来您只提供了一个年份 "2021" 但没有提供具体的问题或者背景信息。如果您需要关于2021年的信息，比如历史事件、科技发展、文化动态等，请提供更多的上下文或者具体问题，我将很乐意为您提供相关信息或者解答疑问。

## 連續交談測試

以下是使用 chat 設計的聊天程式：

In [ ]:
empty_history()
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '使用繁體中文的小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat(sys_msg, msg,
                      stream=True,
                      debug=True,
                      model='gpt-4-1106-preview',
                      ):
        print(reply, end = "")
    print('\n')

你希望ㄟ唉扮演：

你說：2022 金馬獎影后是誰？他執導的電影有哪些？
使用繁體中文的小助理：嘗試叫用：google_res(**{'user_msg': '2022金馬獎影後'})
嘗試叫用：google_res(**{'user_msg': '張艇姊導演的電影'})
2022年的金馬獎影后是張艾嘉，她以電影《燈火闌珊》奪得該獎項。

張艾嘉曾執導的電影包括《相愛相親》。由於提供的信息有限，如果您想了解更多關於張艾嘉執導的電影列表，可能需要更進一步的搜索或查詢。

你說：


## 新增工具函式

以文字生圖為例

用 Image API 設計一個文生圖的工具函式：

In [ ]:
def txt_to_img_url(prompt):
    response = openai.images.generate(
        prompt=prompt,
        n=1,
        size='1024x1024',
        style='vivid',
        quality='hd'
    )
    return response.data[0].url

在工具函式表中新增項目, 生圖後不需要再送回給 AI 處理, 所以 chain 項目設為 False：

In [ ]:
tools_table.append(
    {                    # 每個元素代表一個函式
        "chain": False,  # 生圖後不需要傳回給 API
        "func": txt_to_img_url,
        "spec": {        # function calling 需要的函式規格
            'type': 'function',
            'function': {
                "name": "txt_to_img_url",
                "description": "可由文字生圖並傳回圖像網址",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "prompt": {
                            "type": "string",
                            "description": "描述要產生圖像內容的文字",
                        }
                    },
                    "required": ["prompt"],
                },
            }
        }
    }
)

In [ ]:
tools_table.pop()
len(tools_table)

1

測試看看是不是可以正確生圖？

In [ ]:
for chunk in chat('小助理', '我想要夕陽下海豚躍出海面的圖像', True):
    print(chunk)

嘗試叫用：txt_to_img_url(**{'prompt': 'sunset with dolphin jumping out of the sea'})
https://oaidalleapiprodscus.blob.core.windows.net/private/org-TnN5jDJWh2Gbe6gZ6C11q1fl/user-hwS8wMY6Z8ZzjiE3tcFcl4mM/img-4VnFT09UiHTJ3bEXnUmj4cOH.png?st=2023-12-05T04%3A41%3A57Z&se=2023-12-05T06%3A41%3A57Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-12-04T23%3A04%3A53Z&ske=2023-12-05T23%3A04%3A53Z&sks=b&skv=2021-08-06&sig=KiMHVpzS%2B3FY55ZXz1Ze/A%2BwZh3IxAvEghrsb9kvZu8%3D
